In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import sys

In [7]:
url = "https://www.indeed.co.in/jobs?q=civil+engineering&l=India"

In [8]:
r = requests.get(url)
p = bs(r.text)

In [9]:
def totextfile(filename, title, skills, des):
    exists = os.path.isfile("./" + filename + ".txt")
    if exists:
        print("Job present in directory.")
        return      
    else:
        file = filename + '.txt'
        file1 = open(file,'wb')
        file1.write(title.encode('utf8'))
        file1.write("\n".encode('utf8'))
        for i in skills:
            file1.write(i.encode('utf8'))
            file1.write("\n".encode('utf8'))
        for i in des:
            file1.write(i.encode('utf8'))
            file1.write("\n".encode('utf8'))
        file1.close()
        print(filename, " Done")


In [18]:
def jobtextadd(pag, count, titles):
    for i in pag:
        c = 'https://www.indeed.co.in'+i['href']
        r2 = requests.get(c)
        p1 = bs(r2.text)
        page = p1.find_all('a', {'class':'jobtitle'})
        for i in page:
            c = 'https://www.indeed.co.in' + i['href']
            r1 = requests.get(c)
            r1.url
            page1 = bs(r1.text)
            tit = page1.find('div', {'class' : 'jobsearch-JobInfoHeader-title-container'})
            name = page1.find('div', {'class' : 'jobsearch-CompanyInfoWithoutHeaderImage'})
            title = tit.text
            filename = title + ' ' + name.text
            filename = filename.replace('/','').replace('|','').replace('""','')
            if filename in titles:
                print("Job already present.\n")
                continue
            else:
                skill = page1.find_all('span',{'class' : 'jobsearch-JobMetadataHeader-skillItem'})
                description = page1.find_all('div', {'class' : 'jobsearch-jobDescriptionText'})
                skills = []
                des = []
                for j in skill:
                    skills.append(j.text)
                for k in description:
                    des.append(k.text)
                totextfile(filename,title,skills,des)
                count += 1
                #print(count)
        pages = p1.find_all('div',{'class':'pagination'})
        pag1 = pages[0].find_all('a')
        for p in pag1:
            if p in pag:
                continue
            else:
                pag.append(p)
        if count >= 100:
            print("Stopping!")
            break
        print (i," done")
        

In [19]:
pages = p.find_all('div',{'class':'pagination'})
pag = set(pages[0].find_all('a'))
pag = list(pag)
count = 0
titles = []
jobtextadd(pag,count,titles) 

Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AyDUXvIbtIMLsYoak0xyD2czxZai7hkMhNco0btN9NWrj59XrNQ9gMF9yTotYlw-J81h0S7Ivq2zPXHIlYEiQnkz8GZCNQUVmWhiX490G3OTkaH4HH_Qu0rlqj4uIC1npbd55fVpJCRkgh7k1F9vkG9JVrX-e5JlzyLx_bg69HR97FDwNt9jWXpCmRJRZsZIaXBd4VYYvvDwViDbSX8Nd5opMj76eh2MZwf-5cbJPpWPsa8rDxyneKw_VxB3KVQzileIHrCYRlal1d1S2bq5xWyqKhdnlUVwivuvOs8buAtUJcbsVkBOA5Wz43a8dlvdfuY2vpcYpbafhNjEStIeEIJuhAIBtr2y1AqaDwsx9VIEE3gWe0X3UiTJMIsbQ2B2XCy1FJaAH4SOJDYA48ApJyDstkki86EeciBmPxA6qPoBqxHjwN9bJN8vMWz03WlFpOGF6P37vWUA==&amp;p=15&amp;fvj=1&a